# Ingest Tabular Data

When ingesting structured data from an existing S3 bucket into a SageMaker Notebook, there are multiple ways to handle it. We will introduce the following methods to access your data from the notebook:

* Copying your data to your instance. If you are dealing with a normal size of data or are simply experimenting, you can copy the files into the SageMaker instance and just use it as a file system in your local machine. 
* Using Python packages to directly access your data without copying it. One downside of copying your data to your instance is: if you are done with your notebook instance and delete it, all the data is gone with it unless you store it elsewhere. We will introduce several methods to solve this problem in this notebook, and using python packages is one of them. Also, if you have large data sets (for example, with millions of rows), you can directly read data from S3 utilizing S3 compatible python libraries with built-in functions.
* Using AWS native methods to directly access your data. You can also use AWS native packages like `s3fs` and `aws data wrangler` to access your data directly.  

We will demonstrate how to ingest the a tabular (structured) dataset into a notebook for further analysis. 

## Download data from online resources and write data to S3
In this step, you will download a small sample tabular data and upload it to the default bucket associated with your account. This step mimics preliminary data preparation in a normal ML workflow, i.e. getting data from various sauces and put them into a data lake. You will later ingest the stored data in S3 into your SageMaker Notebook instances. 

In [2]:
%pip install -qU 'sagemaker>=2.15.0' 's3fs==0.4.2' 'awswrangler==1.2.0'
# you would need s3fs version > 0.4.0 for aws data wrangler to work correctly

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import awswrangler as wr
import pandas as pd
import s3fs
import sagemaker

In [4]:
# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
s3 = sagemaker_session.boto_session.resource('s3')
bucket = sagemaker_session.default_bucket() #replace with your own bucket name if you have one
prefix = 'data'
filename = 'sample_tabular_data.csv'

In [5]:
#helper functions to upload data to s3
def write_to_s3(filename, bucket, prefix):
    #put one file in a separate folder. This is helpful if you read and prepare data with Athena
    #filename_key = filename.split('.')[0]
    key = "{}/{}".format(prefix,filename)
    return s3.Bucket(bucket).upload_file(filename,key)

def upload_to_s3(bucket, prefix, filename):
    url = 's3://{}/{}/{}'.format(bucket, prefix, filename)
    print('Writing to {}'.format(url))
    write_to_s3(filename, bucket, prefix)

In [6]:
# download a synthetic tabular data
!wget -q https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/master/synthetic_data/sample_tabular_data.csv

In [7]:
# upload the sample data to S3 
upload_to_s3(bucket, prefix, filename)

Writing to s3://sagemaker-us-east-1-688520471316/data/sample_tabular_data.csv


## Ingest Tabular Data from S3 bucket
### Method 1: Copying data to the Instance
You can use AWS Command Line Interface (CLI) to copy your data from s3 to your SageMaker instance and copy files between your S3 buckets. This is a quick and easy approach when you are dealing with medium-sized data files, or you are experimenting and doing exploratory analysis. The documentation can be found [here](https://docs.aws.amazon.com/cli/latest/reference/s3/cp.html).

In [8]:
#copy data to your sagemaker instance using AWS CLI
!aws s3 cp s3://$bucket/$prefix/$filename $prefix/

download: s3://sagemaker-us-east-1-688520471316/data/sample_tabular_data.csv to data/sample_tabular_data.csv


In [9]:
data_location = "{}/{}".format(prefix, filename)
tabular_data = pd.read_csv(data_location, nrows = 5)
tabular_data.head()

,A,B,C,D,E,F,G,H,I,J,K,L,M,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Method 2: Use AWS compatible Python Packages
When you are dealing with large data sets, or do not want to lose any data when you delete your SageMaker Notebook Instance, you can use pre-built packages to access your files in S3 without copying files into your instance. These packages, such as `Pandas`, have implemented options to access data with a specified path string: while you will use `file://` on your local file system, you will use `s3://` instead to access the data through the AWS boto library. For `pandas`, any valid string path is acceptable. The string could be a URL. Valid URL schemes include http, ftp, s3, and file. For file URLs, a host is expected.You can find additional documentation [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html). 

In [10]:
data_s3_location = "s3://{}/{}/{}".format(bucket, prefix, filename) # S3 URL
s3_tabular_data = pd.read_csv(data_s3_location, nrows = 5)
s3_tabular_data.head()

,A,B,C,D,E,F,G,H,I,J,K,L,M,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Method 3: Use AWS native methods
#### 3.1 s3fs 

[S3Fs](https://s3fs.readthedocs.io/en/latest/) is a Pythonic file interface to S3. It builds on top of botocore. The top-level class S3FileSystem holds connection information and allows typical file-system style operations like cp, mv, ls, du, glob, etc., as well as put/get of local files to/from S3. 

In [11]:
fs = s3fs.S3FileSystem()
data_s3fs_location = "s3://{}/{}/".format(bucket, prefix)
# To List all files in your accessible bucket
fs.ls(data_s3fs_location)

['sagemaker-us-east-1-688520471316/data/DEMO-mnist',
 'sagemaker-us-east-1-688520471316/data/abalone_train.csv',
 'sagemaker-us-east-1-688520471316/data/abalone_valid.csv',
 'sagemaker-us-east-1-688520471316/data/mnist',
 'sagemaker-us-east-1-688520471316/data/sample_tabular_data.csv']

In [12]:
# open it directly with s3fs
data_s3fs_location = "s3://{}/{}/{}".format(bucket, prefix, filename) # S3 URL
with fs.open(data_s3fs_location) as f:
    print(pd.read_csv(f, nrows = 5))

         A     B     C    D      E      F     G       H    I      J     K  \
0  0.00632  18.0  2.31  0.0  0.538  6.575  65.2  4.0900  1.0  296.0  15.3   
1  0.02731   0.0  7.07  0.0  0.469  6.421  78.9  4.9671  2.0  242.0  17.8   
2  0.02729   0.0  7.07  0.0  0.469  7.185  61.1  4.9671  2.0  242.0  17.8   
3  0.03237   0.0  2.18  0.0  0.458  6.998  45.8  6.0622  3.0  222.0  18.7   
4  0.06905   0.0  2.18  0.0  0.458  7.147  54.2  6.0622  3.0  222.0  18.7   

        L     M  target  
0  396.90  4.98    24.0  
1  396.90  9.14    21.6  
2  392.83  4.03    34.7  
3  394.63  2.94    33.4  
4  396.90  5.33    36.2  


#### 3.2 AWS Data Wrangler
[AWS Data Wrangler](https://github.com/awslabs/aws-data-wrangler) is an open-source Python library that extends the power of the Pandas library to AWS connecting DataFrames and AWS data related services (Amazon Redshift, AWS Glue, Amazon Athena, Amazon EMR, Amazon QuickSight, etc), which we will cover in later sections. It is built on top of other open-source projects like Pandas, Apache Arrow, Boto3, s3fs, SQLAlchemy, Psycopg2 and PyMySQL, and offers abstracted functions to execute usual ETL tasks like load/unload data from Data Lakes, Data Warehouses and Databases. Note that you would need `s3fs version > 0.4.0` for the `awswrangler csv reader` to work.

In [13]:
data_wr_location = "s3://{}/{}/{}".format(bucket, prefix, filename) # S3 URL
wr_data = wr.s3.read_csv(path=data_wr_location, nrows = 5)
wr_data.head()

,A,B,C,D,E,F,G,H,I,J,K,L,M,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Citation
Boston Housing data,  Harrison, D. and Rubinfeld, D.L. `Hedonic prices and the demand for clean air', J. Environ. Economics & Management, vol.5, 81-102, 1978.